In [45]:
import models
import pandas as pd

In [46]:
df = models.npy_to_bowpy('copper/test/predictions/test/CNN_32_filters_3_kernels_predictions.csv', 'copper/test/predictions/test/')
df

,pred,corr,true_positive,false_positive,rule_result0,rule_result1,rule_result2,rule_result3,rule_result4,rule_result5,...,rule_result24,rule_result25,rule_result26,rule_result27,rule_result28,rule_result29,rule_result30,rule_result31,rule_result32,rule_result33
0,1,0,0,1,1,1,1,1,1,1,...,0,1,1,0,0,0,0,0,0,1
1,1,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,0,0,0,0,0,0,0,1,0,0,...,0,1,0,1,1,0,0,0,0,0
218,1,0,0,1,1,0,0,1,1,1,...,0,1,0,1,1,1,0,0,1,0
219,1,0,0,1,1,0,1,1,1,1,...,0,1,0,1,1,1,0,0,1,0
220,1,0,0,1,1,0,0,1,0,1,...,0,1,0,1,1,1,0,0,0,0


In [47]:
def calculate_metrics(df, rule_result_column):
    TP = ((df[rule_result_column] == 1) & (df['corr'] == 1)).sum()
    FP = ((df[rule_result_column] == 1) & (df['corr'] == 0)).sum()
    FN = ((df[rule_result_column] == 0) & (df['corr'] == 1)).sum()
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return f1, recall, precision

In [56]:
f1, recall, precision = calculate_metrics(df, "pred")
print(f'Base model results\nf1: {f1}\nrecall: {recall}\n precision: {precision}')

Base model results
f1: 0.2384105960264901
recall: 0.5142857142857142
 precision: 0.15517241379310345


In [48]:
# Calculate metrics for each rule_result column and store them
metrics = []
for i in range(34):
    rule_result_column = f'rule_result{i}'
    f1, recall, precision = calculate_metrics(df, rule_result_column)
    metrics.append((rule_result_column, f1, recall, precision))

# Convert to DataFrame for easier handling
metrics_df = pd.DataFrame(metrics, columns=['Rule', 'F1', 'Recall', 'Precision'])

# Sort and select top 5 for each metric
top_5_f1 = metrics_df.nlargest(5, 'F1')
top_5_recall = metrics_df.nlargest(5, 'Recall')
top_5_precision = metrics_df.nlargest(5, 'Precision')

# top_5_f1, top_5_recall, top_5_precision

In [49]:
def remove_base(df, column):
    return not df[column].equals(df['pred'])

In [50]:
# Get the names of top 5 columns for each metric
top_5_f1_columns = top_5_f1['Rule'].tolist()
top_5_recall_columns = top_5_recall['Rule'].tolist()
top_5_precision_columns = top_5_precision['Rule'].tolist()

# Don't include base in rule
filtered_top_5_f1_columns = [col for col in top_5_f1_columns if remove_base(df, col)]
filtered_top_5_recall_columns = [col for col in top_5_recall_columns if remove_base(df, col)]
filtered_top_5_precision_columns = [col for col in top_5_precision_columns if remove_base(df, col)]

# Define unchanged columns
unchanged_columns = ['pred', 'corr', 'true_positive', 'false_positive']

# 3 DFs, based on top metrics
df_top_5_f1 = df[unchanged_columns + top_5_f1_columns]
df_top_5_recall = df[unchanged_columns + top_5_recall_columns]
df_top_5_precision = df[unchanged_columns + top_5_precision_columns]

df_top_5_f1.shape, df_top_5_recall.shape, df_top_5_precision.shape


((222, 9), (222, 9), (222, 9))

In [51]:
# Rules based on top 5 F1
df_top_5_f1.to_numpy().dump('edcr/data/test_f1.npy')

In [52]:
# Rules based on top 5 Recall
df_top_5_recall.to_numpy().dump('edcr/data/test_recall.npy')

In [53]:
# Rules based on top 5 Precision
df_top_5_precision.to_numpy().dump('edcr/data/test_precision.npy')